In [1]:
email_address_to = 'yunzhong_gao@homedepot.com'
project_id = ''
bq_staging_dataset = ''
table_list_commadelim = 'TABLE_1,TABLE_2'
output_bucket = ''
run_date = ''

In [2]:
!pip install certifi
!pip install chardet
!pip install idna
!pip install requests
!pip install urllib3

You should consider upgrading via the 'c:\users\yxg0rzh\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yxg0rzh\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yxg0rzh\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yxg0rzh\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\yxg0rzh\appdata\local\continuum\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

from datetime import datetime
import pytz

import os

import requests # for send_email function
import mimetypes # for send_email function
import base64 # for send_email function`

In [4]:
email_address_list = email_address_to.split(',')
def create_recipient_list(email_address_list):
    recipient_list = [{"emailAddress":{"address":x}} for x in email_address_list]
    return recipient_list
toRecipients = create_recipient_list(email_address_list)

In [5]:
toRecipients

[{'emailAddress': {'address': 'yunzhong_gao@homedepot.com'}}]

In [6]:
table_list = table_list_commadelim.split(',')

In [7]:
def extract_bq_table_to_gcs_bucket(project, snapshot_dataset, bucket, table, date_yyyymmdd):
    client = bigquery.Client(project)
    blob_path = f"{date_yyyymmdd}/{table}.csv"
    destination_uri = f"gs://{bucket}/" + blob_path
    dataset_ref = bigquery.DatasetReference(project, snapshot_dataset)
    table_ref = dataset_ref.table(table)

    extract_job = client.extract_table(table_ref, destination_uri, location="US")
    extract_job.result()  # Waits for job to complete.

    print(f"Exported {project}:{snapshot_dataset}.{table} to {destination_uri}")
    return blob_path


In [8]:
# Extract BQ Tables to GCS Bucket BLOB (Binary Large OBject)
blob_paths = []
for table in table_list:
    print(table)
    blob_paths.append(extract_bq_table_to_gcs_bucket(project = "analytics-ahs-owned-thd",
                                                     snapshot_dataset = "TEST_DS", 
                                                     bucket = "akaboshi", 
                                                     table = table, 
                                                     date_yyyymmdd ="20210319"
                                                    )
                          )
blob_paths

CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT


C:\Users\yxg0rzh\AppData\Local\Continuum\anaconda3\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Exported analytics-ahs-owned-thd:AMPLIFAI.CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT to gs://akaboshi/20210319/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv
MASTER_ROSTER_CAC_BSC_SNAPSHOT
Exported analytics-ahs-owned-thd:AMPLIFAI.MASTER_ROSTER_CAC_BSC_SNAPSHOT to gs://akaboshi/20210319/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv


['20210319/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv',
 '20210319/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv']

In [9]:
def gcs_download_blob(project, gcs_bucket, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket to a specified destination file."""

    storage_client = storage.Client(project)

    bucket = storage_client.bucket(gcs_bucket)

    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

In [10]:
blob_paths

['20210319/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv',
 '20210319/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv']

In [11]:
extract_bq_table_to_gcs_bucket("analytics-ahs-owned-thd", "AMPLIFAI", "akaboshi", "CAC_APPT_CONFIRMATION_CLOSERATE_DAILY_SNAPSHOT", "20210319")

Exported analytics-ahs-owned-thd:AMPLIFAI.CAC_APPT_CONFIRMATION_CLOSERATE_DAILY_SNAPSHOT to gs://akaboshi/20210319/CAC_APPT_CONFIRMATION_CLOSERATE_DAILY_SNAPSHOT.csv


'20210319/CAC_APPT_CONFIRMATION_CLOSERATE_DAILY_SNAPSHOT.csv'

In [12]:
downloaded_file_dir = 'downloaded_files/'
if not os.path.isdir(downloaded_file_dir):
    os.mkdir(downloaded_file_dir)

# assumption for [9:] in next line: start at 10th character; beginning of blob_path is 'YYYYMMDD/'
file_names = [x[9:] for x in blob_paths]
file_paths = [downloaded_file_dir + x for x in file_names]

print('file_paths: ')
print(file_paths)

for (table, blob_path, file_path) in zip(table_list, blob_paths, file_paths):
    gcs_download_blob(project = "analytics-ahs-owned-thd", 
                      gcs_bucket = "akaboshi", 
                      source_blob_name = blob_path, 
                      destination_file_name = file_path
                     )


file_paths: 
['downloaded_files/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv', 'downloaded_files/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv']
Blob 20210319/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv downloaded to downloaded_files/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv.
Blob 20210319/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv downloaded to downloaded_files/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv.


In [13]:
downloaded_file_dir = 'downloaded_files/'
if not os.path.isdir(downloaded_file_dir):
    os.mkdir(downloaded_file_dir)

In [14]:
os.path

<module 'ntpath' from 'C:\\Users\\yxg0rzh\\AppData\\Local\\Continuum\\anaconda3\\lib\\ntpath.py'>

In [15]:
get_user(get_auth_token())

NameError: name 'get_user' is not defined

In [16]:
def get_attachment_list(file_paths):
        attached_files = []
        for file_path, file_name in zip(file_paths, file_names):
            b64_content = base64.b64encode(open(file_path, 'rb').read())
            assert len(b64_content) / (1000000) < 4, file_name + ': Attachments must be < 4 MB each'
#             mime_type = mimetypes.guess_type(filename)[0]
#             mime_type = mime_type if mime_type else ''
            attached_files.append(
                {'@odata.type': '#microsoft.graph.fileAttachment',
                 'name': file_name,
                 'contentType': "text/csv",
                 'contentBytes': b64_content.decode('utf-8')
                }
            )
        return attached_files

In [17]:
get_attachment_list(file_paths)

[{'@odata.type': '#microsoft.graph.fileAttachment',
  'name': 'CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv',
  'contentType': 'text/csv',
  'contentBytes': '...'},
 {'@odata.type': '#microsoft.graph.fileAttachment',
  'name': 'MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv',
  'contentType': 'text/csv',
  'contentBytes': '...'}]

In [18]:
file_paths

['downloaded_files/CAC_CONFORMANCE_DAILY_DATA_SNAPSHOT.csv',
 'downloaded_files/MASTER_ROSTER_CAC_BSC_SNAPSHOT.csv']

In [19]:
def send_email(file_paths, file_names, toRecipients):
    API_VERSION = 'v1.0'

    APP_ID = "d5351b39-a423-4215-857e-77250f6e899f"
    TENANT = "fb7e6711-b619-4fbe-afe6-f83b12673323"
    SECRET = "5ZT1HH=X8t[_[8r3@ardnqoSglo6pA8n"
    SENDER_EMAIL = "Services_Analytics_Support@homedepot.com"

    def get_auth_token():
        """
        POST /{tenant}/oauth2/v2.0/token HTTP/1.1           //Line breaks for clarity
        Host: login.microsoftonline.com
        Content-Type: application/x-www-form-urlencoded

        client_id=
        &scope=https%3A%2F%2Fgraph.microsoft.com%2F.default
        &client_secret=
        &grant_type=client_credentials
        :return:
        """

        AUTH_URL = "https://login.microsoftonline.com/{tenant}/oauth2/v2.0/token"

        payload = {"client_id": APP_ID,
                   "scope": "https://graph.microsoft.com/.default",
                   "client_secret": SECRET,
                   "grant_type": "client_credentials"}

        headers = {'content-type': 'application/x-www-form-urlencoded'}

        response = requests.post(AUTH_URL.format(tenant=TENANT, client_id=APP_ID, secret=SECRET), data=payload, headers=headers)

        return response.json()

    def get_attachment_list(file_paths):
        attached_files = []
        for file_path, file_name in zip(file_paths, file_names):
            b64_content = base64.b64encode(open(file_path, 'rb').read())
            assert len(b64_content) / (1000000) < 4, file_name + ': Attachments must be < 4 MB each'
#             mime_type = mimetypes.guess_type(filename)[0]
#             mime_type = mime_type if mime_type else ''
            attached_files.append(
                {'@odata.type': '#microsoft.graph.fileAttachment',
                 'name': file_name,
                 'contentType': "text/csv",
                 'contentBytes': b64_content.decode('utf-8')
                }
            )
        return attached_files
    
    def send_mail(token, toRecipients):
        """
        Other available options are:

        importance:  Low, Normal, High

        For all other options on the email see:
        https://docs.microsoft.com/en-us/graph/api/resources/message?view=graph-rest-1.0


        """     
        
        body = {
            "message": {
                "importance": "Normal",
                "subject": "THD SOCC Scorecard",
                "body": {
                    "contentType": "Text",
                    "content": ""
                },
                "toRecipients": toRecipients,
                "attachments": get_attachment_list(file_paths)
            },
            "saveToSentItems": "true"
        }

        headers = {'Authorization': '{token_type} {token}'.format(token_type=token.get('token_type'), token=token.get('access_token')),
                   "Content-type": "application/json"}

        GRAPH_URL = f"https://graph.microsoft.com/{API_VERSION}/users/{SENDER_EMAIL}/sendMail"

        response = requests.post(GRAPH_URL, headers=headers, json=body)
        
        return response

    if __name__ == "__main__":
        token_response = get_auth_token()
        print(token_response)
        send_mail_response = send_mail(token_response, toRecipients)
    
    return [token_response, send_mail_response]

In [20]:
email_output = send_email(file_paths = file_paths, file_names = file_names, toRecipients = toRecipients)


{'token_type': 'Bearer', 'expires_in': 3599, 'ext_expires_in': 3599, 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6InNDNUxQVW5Sd0tFMEl4NGI4SUxLUEttTHpfNE1seHhCaDVSbENEeEJzZzgiLCJhbGciOiJSUzI1NiIsIng1dCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyIsImtpZCI6Im5PbzNaRHJPRFhFSzFqS1doWHNsSFJfS1hFZyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9mYjdlNjcxMS1iNjE5LTRmYmUtYWZlNi1mODNiMTI2NzMzMjMvIiwiaWF0IjoxNjE4NTIwMDE3LCJuYmYiOjE2MTg1MjAwMTcsImV4cCI6MTYxODUyMzkxNywiYWlvIjoiRTJaZ1lHZzFpSzc0RTNLTE9aVHQ2TGRLMjRhSEFBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJTZXJ2aWNlQW5hbHl0aWNzU3VwcG9ydCIsImFwcGlkIjoiZDUzNTFiMzktYTQyMy00MjE1LTg1N2UtNzcyNTBmNmU4OTlmIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvZmI3ZTY3MTEtYjYxOS00ZmJlLWFmZTYtZjgzYjEyNjczMzIzLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiOTI3MWFjYzUtMTc1MS00NzQxLThiM2MtNjFhNjBkNzA2MDFiIiwicmgiOiIwLkFRNEFFV2QtLXhtMnZrLXY1dmc3RW1jekl6a2JOZFVqcEJWQ2hYNTNKUTl1aVo4T0FBQS4iLCJyb2xlcyI6WyJNYWlsLlNlbmQiXSwic3ViIjoiOTI3MWFjY

In [21]:
print('Headers:')
print(email_output[1].content)

Headers:
b''


In [22]:
if email_output[1].content == b'':
    print('  No Error: HTTP Status ' + str(email_output[1].status_code))
else:
    print('  ' + str(email_output[1].content))

  No Error: HTTP Status 202


In [23]:
for file in file_paths:
    os.remove(file)
os.rmdir(downloaded_file_dir)